In [1]:
# General Purpose
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import odeint
from scipy.integrate import ode
from scipy import special, integrate
from math import erf

# Jupyter Specifics
from IPython.display import HTML
from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout, interact_manual

%matplotlib inline

style = {'description_width': '150px'}
slider_layout = Layout(width='99%')

In [2]:
def dispcalc_ON(time, Lt, Rt, Q, D, timeSplit):
    
    U = 2*Q/3.14159/Rt**2
    k = Rt**2*U**2/192/D
    if time < timeSplit:
        CONC = 0.5 - 0.5*special.erf(0.5*(Lt - U/2*time)*k**(-0.5)*pow(time,-0.5))
    else:
        ### here we need to shift the time range to start from 0 (time - timeSplit), and also change the equation
        ### describing the dispersion (- changes to a +)
        tt = time - timeSplit
        CONC = 0.5 + 0.5*special.erf(0.5*(Lt - U/2*tt)*k**(-0.5)*pow(tt,-0.5))
    return CONC

In [4]:
def main(C, K1, K2, Bo, timeMax, Lt, Rt, Q, D, timeSplit):
    
    
    Cr = C*1e-14 ### Cr [=] mol/mm3        C [=] nM
    K1r = K1*1e6 ### K1r[=] mm3/mol/s     K1 [=] 1/M/s
    Bor = Bo*1e-14 ### Bor [=] mol/mm2    Bo [=] ng/cm2
    

    def function(AB, time):
        CONC = Cr*dispcalc_ON(time, Lt, Rt, Q, D, timeSplit)
        return CONC*K1r*(Bor - AB) - K2*AB
     

    Kd = K2/K1r
    time = np.linspace(1, timeMax, 3600)

    ### working odeint, but gives strange results sometimes
    solution = odeint(function, 0, time, rtol = 1e-20, atol = 1e-20)
    


    #Graphic details
    fig, ax = plt.subplots(figsize=(15, 10))

    ax.plot(time, solution/Bor, 'k.', label='signal')
   
    FracMax = (Cr)/(Cr + Kd)
    

    ax.set_xlim([0, timeMax])
#     ax.set_ylim([0, 1])
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Fractional Coverage')
    ax.legend(loc='best')
    ax.grid()

    plt.show()


interact(main, C = FloatSlider(min=0.01, max=100, step=0.05, value=40, description='Concentration (nM)', style=style, layout=slider_layout),
               K1 = FloatSlider(min=1000, max=1e6, step=2000, value=3e4, description='K1 (1/M/s)', style=style, layout=slider_layout),
               K2 = FloatSlider(min=1e-4, max=1e-2, step=1e-4, value=1e-3, description='K2 (1/s)', style=style, layout=slider_layout),
               Bo = FloatSlider(min=0.05, max=10, step=0.5, value=1, description='Bo (ng/cm2)', style=style, layout=slider_layout),
               timeMax = IntSlider(min=100, max=3600, step=5, value=3600, description='MaxTime', style=style, layout=slider_layout),
               Lt = 200,
               Rt = 0.29,
               Q = 20/60,
               D = 7e-5,
               timeSplit = 1000
        );

interactive(children=(FloatSlider(value=40.0, description='Concentration (nM)', layout=Layout(width='99%'), mi…